<h1>Pastebin Webscraping: Large Sample Analysis</h1>

In [1]:
import numpy as np
import pandas as pd
import mysql.connector
import getpass
import StringIO
import collections

from flask import Flask, render_template, make_response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from wordcloud import WordCloud
from palettable.colorbrewer.qualitative import Set1_5

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mpld3
from mpld3 import utils, plugins

In [5]:
pswd = getpass.getpass('Password:')

Password:········


## Some Ugly Hacking to Make Clickable Links

In [6]:
import json
import uuid
import matplotlib
import os
import warnings
import re

class PluginBase(object):
    def get_dict(self):
        return self.dict_

    def javascript(self):
        if hasattr(self, "JAVASCRIPT"):
            if hasattr(self, "js_args_"):
                return self.JAVASCRIPT.render(self.js_args_)
            else:
                return self.JAVASCRIPT
        else:
            return ""

    def css(self):
        if hasattr(self, "css_"):
            return self.css_
        else:
            return ""
        
#            obj.elements().on("mouseover", function(d) {tooltip.html(labels[i]).style("visibility", "visible");});
class ClickInfo(plugins.PluginBase):
    """Plugin for getting info on click"""

    JAVASCRIPT = """
    mpld3.register_plugin("clickinfo", ClickInfo);
    ClickInfo.prototype = Object.create(mpld3.Plugin.prototype);
    ClickInfo.prototype.constructor = ClickInfo;
    ClickInfo.prototype.requiredProps = ["ids","targets"];
    
    function ClickInfo(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };


    ClickInfo.prototype.draw = function(){
        var targets = this.props.targets;
        
        this.props.ids.forEach(function(id, i) {
            var obj = mpld3.get_element(id);
            obj.elements().on("mousedown", function(d) {window.open(targets[i]);});            
        });
    }
    """    

    def __init__(self, bars, targets):
        self.targets = targets
        
        self.dict_ = {"type": "clickinfo",
                      "ids": [utils.get_id(bar) for bar in bars],
                      "targets": targets} 
    
class PointClickableHTMLTooltip_Changed(PluginBase):
    JAVASCRIPT="""
    mpld3.register_plugin("clickablehtmltooltip", PointClickableHTMLTooltip);
    PointClickableHTMLTooltip.prototype = Object.create(mpld3.Plugin.prototype);
    PointClickableHTMLTooltip.prototype.constructor = PointClickableHTMLTooltip;
    PointClickableHTMLTooltip.prototype.requiredProps = ["id"];
    PointClickableHTMLTooltip.prototype.defaultProps = {labels:null,
                                                 targets:null,
                                                 hoffset:0,
                                                 voffset:10};
    function PointClickableHTMLTooltip(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };
        
    PointClickableHTMLTooltip.prototype.draw = function(){
       var obj = mpld3.get_element(this.props.id);
       var labels = this.props.labels;
       var targets = this.props.targets;
       var tooltip = d3.select("body").append("div")
                    .attr("class", "mpld3-tooltip")
                    .style("position", "absolute")
                    .style("z-index", "10")
                    .style("visibility", "hidden");
       obj.elements()
           .on("mouseover", function(d, i){
                  if ($(obj.elements()[0][0]).css( "fill-opacity" ) > 0 || $(obj.elements()[0][0]).css( "stroke-opacity" ) > 0) {
                              tooltip.html(labels[i])
                                     .style("visibility", "visible");
                              } })
           .on("mousedown", function(d, i){
                              window.open(targets[i]);
                               })
           .on("mousemove", function(d, i){
                  tooltip
                    .style("top", d3.event.pageY + this.props.voffset + "px")
                    .style("left",d3.event.pageX + this.props.hoffset + "px");
                 }.bind(this))
           .on("mouseout",  function(d, i){
                           tooltip.style("visibility", "hidden");});
    };
    """
    def __init__(self, points, labels=None, targets=None,
                 hoffset=2, voffset=-6, css=None):
        self.points = points
        self.labels = labels
        self.targets = targets
        self.voffset = voffset
        self.hoffset = hoffset
        self.css_ = css or ""
        if targets is not None:
            styled_targets = map(lambda x: self.css_ + x, targets)
        else:
            styled_targets = None


        if isinstance(points, matplotlib.lines.Line2D):
            suffix = "pts"
        else:
            suffix = None
        self.dict_ = {"type": "clickablehtmltooltip",
                      "id": utils.get_id(points, suffix),
                      "labels": labels,
                      "targets": styled_targets,
                      "hoffset": hoffset,"voffset": voffset}

In [ ]:
from signal import signal, SIGPIPE, SIG_DFL
signal(SIGPIPE, SIG_DFL) 

app = Flask(__name__)

@app.route('/cluster1b')
def cluster1b():
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconwords1")
        cluster = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    clusterb = []

    for i in cluster:
        clusterb.append(i[0].encode('ascii','ignore'))

    counts = dict(collections.Counter(clusterb).most_common(20))
    labels, values = zip(*counts.items())

    indSort = np.argsort(values)[::-1]
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]
    indexes = np.arange(len(labels))

    fig = Figure(figsize=(12,12), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    wordcloud = WordCloud(background_color="white", width=500, height=500, max_words=300, relative_scaling=.9, prefer_horizontal=.7, collocations=False).generate(str(clusterb))
    axis.imshow(wordcloud, interpolation='bilinear')
    axis.set_title("Word Cloud", size=40, y=1.02, fontweight='bold')
    axis.set_axis_off()
    
    canvas = FigureCanvas(fig)
    output = StringIO.StringIO()
    canvas.print_png(output)
    response = make_response(output.getvalue())
    response.mimetype = 'image/png'

    return response

@app.route('/cluster2b')
def cluster2b():
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconwords2")
        cluster = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    clusterb = []

    for i in cluster:
        clusterb.append(i[0].encode('ascii','ignore'))

    counts = dict(collections.Counter(clusterb).most_common(20))
    labels, values = zip(*counts.items())

    indSort = np.argsort(values)[::-1]
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]
    indexes = np.arange(len(labels))

    fig = Figure(figsize=(12,12), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    wordcloud = WordCloud(background_color="white", width=500, height=500, max_words=300, relative_scaling=.9, prefer_horizontal=.7, collocations=False).generate(str(clusterb))
    axis.imshow(wordcloud, interpolation='bilinear')
    axis.set_title("Word Cloud", size=40, y=1.02, fontweight='bold')
    axis.set_axis_off()
    
    canvas = FigureCanvas(fig)
    output = StringIO.StringIO()
    canvas.print_png(output)
    response = make_response(output.getvalue())
    response.mimetype = 'image/png'

    return response

@app.route('/cluster3b')
def cluster3b():
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconwords3")
        cluster = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    clusterb = []

    for i in cluster:
        clusterb.append(i[0].encode('ascii','ignore'))

    counts = dict(collections.Counter(clusterb).most_common(20))
    labels, values = zip(*counts.items())

    indSort = np.argsort(values)[::-1]
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]
    indexes = np.arange(len(labels))

    fig = Figure(figsize=(12,12), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    wordcloud = WordCloud(background_color="white", width=500, height=500, max_words=300, relative_scaling=.9, prefer_horizontal=.7, collocations=False).generate(str(clusterb))
    axis.imshow(wordcloud, interpolation='bilinear')
    axis.set_title("Word Cloud", size=40, y=1.02, fontweight='bold')
    axis.set_axis_off()
    
    canvas = FigureCanvas(fig)
    output = StringIO.StringIO()
    canvas.print_png(output)
    response = make_response(output.getvalue())
    response.mimetype = 'image/png'

    return response

@app.route('/cluster4b')
def cluster4b():
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconwords4")
        cluster = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    clusterb = []

    for i in cluster:
        clusterb.append(i[0].encode('ascii','ignore'))

    counts = dict(collections.Counter(clusterb).most_common(20))
    labels, values = zip(*counts.items())

    indSort = np.argsort(values)[::-1]
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]
    indexes = np.arange(len(labels))

    fig = Figure(figsize=(12,12), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    wordcloud = WordCloud(background_color="white", width=500, height=500, max_words=300, relative_scaling=.9, prefer_horizontal=.7, collocations=False).generate(str(clusterb))
    axis.imshow(wordcloud, interpolation='bilinear')
    axis.set_title("Word Cloud", size=40, y=1.02, fontweight='bold')
    axis.set_axis_off()
    
    canvas = FigureCanvas(fig)
    output = StringIO.StringIO()
    canvas.print_png(output)
    response = make_response(output.getvalue())
    response.mimetype = 'image/png'

    return response

@app.route('/cluster5b')
def cluster5b():
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconwords5")
        cluster = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    clusterb = []

    for i in cluster:
        clusterb.append(i[0].encode('ascii','ignore'))

    counts = dict(collections.Counter(clusterb).most_common(20))
    labels, values = zip(*counts.items())

    indSort = np.argsort(values)[::-1]
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]
    indexes = np.arange(len(labels))

    fig = Figure(figsize=(12,12), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    wordcloud = WordCloud(background_color="white", width=500, height=500, max_words=300, relative_scaling=.9, prefer_horizontal=.7, collocations=False).generate(str(clusterb))
    axis.imshow(wordcloud, interpolation='bilinear')
    axis.set_title("Word Cloud", size=40, y=1.02, fontweight='bold')
    axis.set_axis_off()
    
    canvas = FigureCanvas(fig)
    output = StringIO.StringIO()
    canvas.print_png(output)
    response = make_response(output.getvalue())
    response.mimetype = 'image/png'

    return response

@app.route('/cluster1')
def cluster1():
    pic1 = "http://localhost:8080/cluster1b"
    pic2 = "http://localhost:8080/cluster1a"
    
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT TEXT FROM pastebindsconclusters1 ORDER BY RAND() LIMIT 1")
        text = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()       
        
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconclusters1")
        scraped = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()
               
    scraped_df = pd.DataFrame(scraped, columns=["LINK","TIME","TEXT"])
    scraped_df["REAL_TIME"] = scraped_df["TIME"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))    

    hist_data = scraped_df["REAL_TIME"].value_counts()
    hist_sort = hist_data.sort_index()

    axismin = max(hist_sort.axes[0]) - timedelta(hours=1)
    axismax = max(hist_sort.axes[0])

    #Set divs to change appearance
    divs = 10

    #ticks creator
    gaps = 60/divs
    ticks = []
    labels = []
    for i in range(divs+1):
        ticks.append(axismin+timedelta(minutes=(i*gaps)))
        labels.append((axismin+timedelta(minutes=(i*gaps))).time())
    
    fig = Figure(figsize=(9,9), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    axis.plot_date(hist_sort.axes[0], hist_sort, linestyle='solid', marker='None', color=Set1_5.hex_colors[0])
    
    for y in range(0, max(hist_sort)+5, 5):
        axis.plot_date([axismin, axismax], [y,y], "--", lw=0.5, color="black", alpha=0.7) 

    axis.set_xlim(axismin, axismax)
    axis.set_xticks(ticks, labels)
    axis.set_ylim(0, max(hist_sort)+5)
    axis.tick_params(axis='y', length=0, **{"labelsize":"x-large"})
    axis.tick_params(axis='x', length=0, **{"labelsize":"medium"})
    
    axis.spines["top"].set_visible(False)    
    axis.spines["bottom"].set_visible(False)    
    axis.spines["right"].set_visible(False)    
    axis.spines["left"].set_visible(False)
    
    labels2 = []
    for i in hist_sort:
        labels2.append(str(i))
    
    points = axis.plot_date(hist_sort.axes[0], hist_sort, 'o', color=Set1_5.hex_colors[0], ms=7, mew=0, alpha=.6)
    cluster_number = 1
        
    targets = []
    for i in hist_sort.axes[0]:
        targets.append("http://localhost:8080/time_search_c/" + str(cluster_number) + "-" + str(i).replace(" ","-").replace(":","-")[:-3])
    
    tooltip = PointClickableHTMLTooltip_Changed(points[0], labels2, targets=targets, voffset=10, hoffset=10)
    plugins.connect(fig, tooltip)
    
    return render_template("cluster_page.html", **{"pic1":pic1, "pic2":pic2, "text":text[0][0], "script":mpld3.fig_to_html(fig),"n":len(scraped_df),"examples":list(scraped_df["LINK"][-5:]),"cluster_number":cluster_number})

@app.route('/cluster2')
def cluster2():
    pic1 = "http://localhost:8080/cluster2b"
    pic2 = "http://localhost:8080/cluster2a"
    
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT TEXT FROM pastebindsconclusters2 ORDER BY RAND() LIMIT 1")
        text = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconclusters2")
        scraped = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()
               
    scraped_df = pd.DataFrame(scraped, columns=["LINK","TIME","TEXT"])
    scraped_df["REAL_TIME"] = scraped_df["TIME"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))    

    hist_data = scraped_df["REAL_TIME"].value_counts()
    hist_sort = hist_data.sort_index()

    axismin = max(hist_sort.axes[0]) - timedelta(hours=1)
    axismax = max(hist_sort.axes[0])

    #Set divs to change appearance
    divs = 10

    #ticks creator
    gaps = 60/divs
    ticks = []
    labels = []
    for i in range(divs+1):
        ticks.append(axismin+timedelta(minutes=(i*gaps)))
        labels.append((axismin+timedelta(minutes=(i*gaps))).time())
    
    fig = Figure(figsize=(9,9), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    axis.plot_date(hist_sort.axes[0], hist_sort, linestyle='solid', marker='None', color=Set1_5.hex_colors[1])
    
    for y in range(0, max(hist_sort)+5, 5):
        axis.plot_date([axismin, axismax], [y,y], "--", lw=0.5, color="black", alpha=0.7) 

    axis.set_xlim(axismin, axismax)
    axis.set_xticks(ticks, labels)
    axis.set_ylim(0, max(hist_sort)+5)
    axis.tick_params(axis='y', length=0, **{"labelsize":"x-large"})
    axis.tick_params(axis='x', length=0, **{"labelsize":"medium"})
    
    axis.spines["top"].set_visible(False)    
    axis.spines["bottom"].set_visible(False)    
    axis.spines["right"].set_visible(False)    
    axis.spines["left"].set_visible(False)
    
    labels2 = []
    for i in hist_sort:
        labels2.append(str(i))
    
    points = axis.plot_date(hist_sort.axes[0], hist_sort, 'o', color=Set1_5.hex_colors[0], ms=7, mew=0, alpha=.6)
    cluster_number = 2
        
    targets = []
    for i in hist_sort.axes[0]:
        targets.append("http://localhost:8080/time_search_c/" + str(cluster_number) + "-" + str(i).replace(" ","-").replace(":","-")[:-3])
    
    tooltip = PointClickableHTMLTooltip_Changed(points[0], labels2, targets=targets, voffset=10, hoffset=10)
    plugins.connect(fig, tooltip)
    
    return render_template("cluster_page.html", **{"pic1":pic1, "pic2":pic2, "text":text[0][0], "script":mpld3.fig_to_html(fig),"n":len(scraped_df),"examples":list(scraped_df["LINK"][-5:]),"cluster_number":cluster_number})

@app.route('/cluster3')
def cluster3():
    pic1 = "http://localhost:8080/cluster3b"
    pic2 = "http://localhost:8080/cluster3a"
    
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT TEXT FROM pastebindsconclusters3 ORDER BY RAND() LIMIT 1")
        text = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconclusters3")
        scraped = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()
               
    scraped_df = pd.DataFrame(scraped, columns=["LINK","TIME","TEXT"])
    scraped_df["REAL_TIME"] = scraped_df["TIME"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))    

    hist_data = scraped_df["REAL_TIME"].value_counts()
    hist_sort = hist_data.sort_index()

    axismin = max(hist_sort.axes[0]) - timedelta(hours=1)
    axismax = max(hist_sort.axes[0])

    #Set divs to change appearance
    divs = 10

    #ticks creator
    gaps = 60/divs
    ticks = []
    labels = []
    for i in range(divs+1):
        ticks.append(axismin+timedelta(minutes=(i*gaps)))
        labels.append((axismin+timedelta(minutes=(i*gaps))).time())
    
    fig = Figure(figsize=(9,9), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    axis.plot_date(hist_sort.axes[0], hist_sort, linestyle='solid', marker='None', color=Set1_5.hex_colors[2])
    
    for y in range(0, max(hist_sort)+5, 5):
        axis.plot_date([axismin, axismax], [y,y], "--", lw=0.5, color="black", alpha=0.7) 

    axis.set_xlim(axismin, axismax)
    axis.set_xticks(ticks, labels)
    axis.set_ylim(0, max(hist_sort)+5)
    axis.tick_params(axis='y', length=0, **{"labelsize":"x-large"})
    axis.tick_params(axis='x', length=0, **{"labelsize":"medium"})
    
    axis.spines["top"].set_visible(False)    
    axis.spines["bottom"].set_visible(False)    
    axis.spines["right"].set_visible(False)    
    axis.spines["left"].set_visible(False)
    
    labels2 = []
    for i in hist_sort:
        labels2.append(str(i))
    
    points = axis.plot_date(hist_sort.axes[0], hist_sort, 'o', color=Set1_5.hex_colors[0], ms=7, mew=0, alpha=.6)
    cluster_number = 3
        
    targets = []
    for i in hist_sort.axes[0]:
        targets.append("http://localhost:8080/time_search_c/" + str(cluster_number) + "-" + str(i).replace(" ","-").replace(":","-")[:-3])
    
    tooltip = PointClickableHTMLTooltip_Changed(points[0], labels2, targets=targets, voffset=10, hoffset=10)
    plugins.connect(fig, tooltip)
    
    return render_template("cluster_page.html", **{"pic1":pic1, "pic2":pic2, "text":text[0][0], "script":mpld3.fig_to_html(fig),"n":len(scraped_df),"examples":list(scraped_df["LINK"][-5:]),"cluster_number":cluster_number})

@app.route('/cluster4')
def cluster4():
    pic1 = "http://localhost:8080/cluster4b"
    pic2 = "http://localhost:8080/cluster4a"
    
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT TEXT FROM pastebindsconclusters4 ORDER BY RAND() LIMIT 1")
        text = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconclusters4")
        scraped = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()
               
    scraped_df = pd.DataFrame(scraped, columns=["LINK","TIME","TEXT"])
    scraped_df["REAL_TIME"] = scraped_df["TIME"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))    

    hist_data = scraped_df["REAL_TIME"].value_counts()
    hist_sort = hist_data.sort_index()

    axismin = max(hist_sort.axes[0]) - timedelta(hours=1)
    axismax = max(hist_sort.axes[0])

    #Set divs to change appearance
    divs = 10

    #ticks creator
    gaps = 60/divs
    ticks = []
    labels = []
    for i in range(divs+1):
        ticks.append(axismin+timedelta(minutes=(i*gaps)))
        labels.append((axismin+timedelta(minutes=(i*gaps))).time())
    
    fig = Figure(figsize=(9,9), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    axis.plot_date(hist_sort.axes[0], hist_sort, linestyle='solid', marker='None', color=Set1_5.hex_colors[3])
    
    for y in range(0, max(hist_sort)+5, 5):
        axis.plot_date([axismin, axismax], [y,y], "--", lw=0.5, color="black", alpha=0.7) 

    axis.set_xlim(axismin, axismax)
    axis.set_xticks(ticks, labels)
    axis.set_ylim(0, max(hist_sort)+5)
    axis.tick_params(axis='y', length=0, **{"labelsize":"x-large"})
    axis.tick_params(axis='x', length=0, **{"labelsize":"medium"})
    
    axis.spines["top"].set_visible(False)    
    axis.spines["bottom"].set_visible(False)    
    axis.spines["right"].set_visible(False)    
    axis.spines["left"].set_visible(False)
    
    labels2 = []
    for i in hist_sort:
        labels2.append(str(i))
    
    points = axis.plot_date(hist_sort.axes[0], hist_sort, 'o', color=Set1_5.hex_colors[0], ms=7, mew=0, alpha=.6)
    cluster_number = 4
        
    targets = []
    for i in hist_sort.axes[0]:
        targets.append("http://localhost:8080/time_search_c/" + str(cluster_number) + "-" + str(i).replace(" ","-").replace(":","-")[:-3])
    
    tooltip = PointClickableHTMLTooltip_Changed(points[0], labels2, targets=targets, voffset=10, hoffset=10)
    plugins.connect(fig, tooltip)
    
    return render_template("cluster_page.html", **{"pic1":pic1, "pic2":pic2, "text":text[0][0], "script":mpld3.fig_to_html(fig),"n":len(scraped_df),"examples":list(scraped_df["LINK"][-5:]),"cluster_number":cluster_number})

@app.route('/cluster5')
def cluster5():
    pic1 = "http://localhost:8080/cluster5b"
    pic2 = "http://localhost:8080/cluster5a"
    
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT TEXT FROM pastebindsconclusters5 ORDER BY RAND() LIMIT 1")
        text = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconclusters5")
        scraped = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()
               
    scraped_df = pd.DataFrame(scraped, columns=["LINK","TIME","TEXT"])
    scraped_df["REAL_TIME"] = scraped_df["TIME"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))    

    hist_data = scraped_df["REAL_TIME"].value_counts()
    hist_sort = hist_data.sort_index()

    axismin = max(hist_sort.axes[0]) - timedelta(hours=1)
    axismax = max(hist_sort.axes[0])

    #Set divs to change appearance
    divs = 10

    #ticks creator
    gaps = 60/divs
    ticks = []
    labels = []
    for i in range(divs+1):
        ticks.append(axismin+timedelta(minutes=(i*gaps)))
        labels.append((axismin+timedelta(minutes=(i*gaps))).time())
    
    fig = Figure(figsize=(9,9), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    axis.plot_date(hist_sort.axes[0], hist_sort, linestyle='solid', marker='None', color=Set1_5.hex_colors[4])
    
    for y in range(0, max(hist_sort)+5, 5):
        axis.plot_date([axismin, axismax], [y,y], "--", lw=0.5, color="black", alpha=0.7) 

    axis.set_xlim(axismin, axismax)
    axis.set_xticks(ticks, labels)
    axis.set_ylim(0, max(hist_sort)+5)
    axis.tick_params(axis='y', length=0, **{"labelsize":"x-large"})
    axis.tick_params(axis='x', length=0, **{"labelsize":"medium"})
    
    axis.spines["top"].set_visible(False)    
    axis.spines["bottom"].set_visible(False)    
    axis.spines["right"].set_visible(False)    
    axis.spines["left"].set_visible(False)
    
    labels2 = []
    for i in hist_sort:
        labels2.append(str(i))
    
    points = axis.plot_date(hist_sort.axes[0], hist_sort, 'o', color=Set1_5.hex_colors[0], ms=7, mew=0, alpha=.6)
    cluster_number = 5
        
    targets = []
    for i in hist_sort.axes[0]:
        targets.append("http://localhost:8080/time_search_c/" + str(cluster_number) + "-" + str(i).replace(" ","-").replace(":","-")[:-3])
    
    tooltip = PointClickableHTMLTooltip_Changed(points[0], labels2, targets=targets, voffset=10, hoffset=10)
    plugins.connect(fig, tooltip)
    
    return render_template("cluster_page.html", **{"pic1":pic1, "pic2":pic2, "text":text[0][0], "script":mpld3.fig_to_html(fig),"n":len(scraped_df),"examples":list(scraped_df["LINK"][-5:]),"cluster_number":cluster_number})

@app.route('/cluster_home')
def cluster_home():
    
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT COUNT(*) FROM pastebindsconlarge")
        all_scans = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()    
    
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconflag")
        flags = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT COUNT(*) FROM pastebindsconflag")
        total = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()        
    
    scripts=[]
    for i in range(5):
        try:
            cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
            cursor = cnx.cursor(buffered=True)
            stmt = "SELECT * FROM pastebindsconwords%s" % (i+1)
            cursor.execute(stmt)
            cluster = cursor.fetchall()        
        except mysql.connector.Error as err:
            print(str(err))
        else:
            cursor.close()
            cnx.close()

        clusterb = []

        for j in cluster:
            clusterb.append(j[0].encode('ascii','ignore'))

        counts = dict(collections.Counter(clusterb).most_common(20))
        labels, values = zip(*counts.items())

        indSort = np.argsort(values)[::-1]
        labels = np.array(labels)[indSort]
        values = np.array(values)[indSort]
        indexes = np.arange(len(labels))
        
        fig = Figure(figsize=(8,8), facecolor="white")
        axis = fig.add_subplot(1, 1, 1)
        axis.bar(indexes, values)
        axis.set_xticks(indexes + 0.35)
        axis.set_xticklabels(labels)
        axis.set_title("Top Word Frequency", size=80, y=1.02, fontweight='bold')

        axis.tick_params(axis='y', length=0, **{"labelsize":"x-large"})
        axis.tick_params(axis='x', length=0, **{"labelsize":"medium"})

        axis.spines["top"].set_visible(False)    
        axis.spines["bottom"].set_visible(False)    
        axis.spines["right"].set_visible(False)    
        axis.spines["left"].set_visible(False)
    
        bars = axis.bar(indexes, values)

        targets = []
        for j in labels:
            targets.append("http://localhost:8080/word_search_c/" + str(i+1) + "-" + str(j.encode('hex')))
    
        plugins.connect(fig, ClickInfo(bars, targets))
        
        scripts.append(mpld3.fig_to_html(fig))
    
    return render_template("home_page.html", **{"flags":flags[-5:],"total":total[0][0],"grand_total":all_scans[0][0]+total[0][0],"scripts":scripts})

@app.route('/pie')
def pie():
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT COUNT(*) FROM pastebindsconclusters1")
        clustercount1 = cursor.fetchall()
        cursor.execute("SELECT COUNT(*) FROM pastebindsconclusters2")
        clustercount2 = cursor.fetchall()
        cursor.execute("SELECT COUNT(*) FROM pastebindsconclusters3")
        clustercount3 = cursor.fetchall()
        cursor.execute("SELECT COUNT(*) FROM pastebindsconclusters4")
        clustercount4 = cursor.fetchall()
        cursor.execute("SELECT COUNT(*) FROM pastebindsconclusters5")
        clustercount5 = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()    

    names=['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5']
    pie_clusters = [clustercount1[0][0],clustercount2[0][0],clustercount3[0][0],clustercount4[0][0],clustercount5[0][0]]

    my_circle=plt.Circle( (0,0), 0.55, color='white')

    fig = Figure(figsize=(10,10), facecolor="white")
    axis = fig.add_subplot(1, 1, 1)
    axis.add_artist(my_circle)
    axis.pie(pie_clusters, labels=names, labeldistance=1.1, wedgeprops = { 'linewidth' : 5, 'edgecolor' : 'white' }, colors=Set1_5.hex_colors)
    for labels in axis.texts:
        labels.set_fontsize(20)

    canvas = FigureCanvas(fig)
    output = StringIO.StringIO()
    canvas.print_png(output)
    response = make_response(output.getvalue())
    response.mimetype = 'image/png'
    
    return response

@app.route('/live_tables')
def live_tables():
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebin")
        to_scrape = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()
   
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        cursor.execute("SELECT * FROM pastebindsconlarge")
        scraped = cursor.fetchall()
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()
               
    scraped_df = pd.DataFrame(scraped, columns=["LINK","TIME","TEXT","TEXT_P"])
    scraped_df["REAL_TIME"] = scraped_df["TIME"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))    

    hist_data = scraped_df["REAL_TIME"].value_counts()
    hist_sort = hist_data.sort_index()

    axismin = max(hist_sort.axes[0]) - timedelta(hours=1)
    axismax = max(hist_sort.axes[0])

    #Set divs to change appearance
    divs = 10

    #ticks creator
    gaps = 60/divs
    ticks = []
    labels = []
    for i in range(divs+1):
        ticks.append(axismin+timedelta(minutes=(i*gaps)))
        labels.append((axismin+timedelta(minutes=(i*gaps))).time())
    
    fig = Figure(figsize=(9,9), facecolor="white", tight_layout=True)
    axis = fig.add_subplot(1, 1, 1)
    axis.plot_date(hist_sort.axes[0], hist_sort, linestyle='solid', marker='None', color=Set1_5.hex_colors[0])
    
    for y in range(0, max(hist_sort)+5, 5):
        axis.plot_date([axismin, axismax], [y,y], "--", lw=0.5, color="black", alpha=0.7) 

    axis.set_xlim(axismin, axismax)
    axis.set_xticks(ticks, labels)
    axis.set_ylim(0, max(hist_sort)+5)
    axis.tick_params(axis='y', length=0, **{"labelsize":"x-large"})
    axis.tick_params(axis='x', length=0, **{"labelsize":"medium"})
    
    axis.spines["top"].set_visible(False)    
    axis.spines["bottom"].set_visible(False)    
    axis.spines["right"].set_visible(False)    
    axis.spines["left"].set_visible(False)
    
    labels2 = []
    for i in hist_sort:
        labels2.append(str(i))
    
    points = axis.plot_date(hist_sort.axes[0], hist_sort, 'o', color=Set1_5.hex_colors[0], ms=7, mew=0, alpha=.6)
    
    targets = []
    for i in hist_sort.axes[0]:
        targets.append("http://localhost:8080/time_search/" + str(i).replace(" ","-").replace(":","-")[:-3])
    
    tooltip = PointClickableHTMLTooltip_Changed(points[0], labels2, targets=targets, voffset=10, hoffset=10)

    plugins.connect(fig, tooltip)
    
    return render_template("live_tables.html", **{"to_scrape":to_scrape, "scraped":scraped[-16:], "scraped_n":len(scraped[-16:]), "script":mpld3.fig_to_html(fig)})

@app.route('/time_search/<time>')
def time_search(time):
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        time_parsed = '%s %s:00' % (time[0:10],time[11:13]+":"+time[14:])
        stmt = "SELECT LINK, TEXT FROM pastebindsconlarge WHERE TIME = '%s'" % (time_parsed)
        cursor.execute(stmt)
        records_raw = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    records = []
    for r in records_raw:
        records.append((r[0],r[1].decode("hex")))
 
    return render_template("event_search.html", **{"records":records,"records_n":len(records),"time":time_parsed})

@app.route('/time_search_c/<time>')
def time_search_c(time):
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        time_parsed = '%s %s:00' % (time[2:12],time[13:15]+":"+time[16:])
        stmt = "SELECT LINK, TEXT FROM pastebindsconclusters%s WHERE TIME = '%s'" % (time[0],time_parsed)
        cursor.execute(stmt)
        records_raw = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    records = []
    for r in records_raw:
        records.append((r[0],r[1]))
 
    return render_template("event_search_c.html", **{"records":records,"records_n":len(records),"time":time_parsed,"cluster":time[0]})

@app.route('/word_search/<search_term>')
def word_search(search_term):
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        stmt = "SELECT LINK, TEXT FROM pastebindsconlarge WHERE TEXT LIKE '%s'" % ("%" + search_term + "%")
        cursor.execute(stmt)
        records_raw = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    records = []
    for r in records_raw:
        records.append((r[0],r[1].decode("hex")))
 
    return render_template("word_search.html", **{"records":records,"records_n":len(records),"word":search_term.decode("hex")})

@app.route('/word_search_c/<search_term>')
def word_search_c(search_term):
    try:
        cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
        cursor = cnx.cursor(buffered=True)
        stmt = "SELECT LINK, TEXT FROM pastebindsconclusters%s WHERE TEXT LIKE '%s'" % (search_term[0],"%" + search_term[2:].decode("hex") + "%")
        cursor.execute(stmt)
        records_raw = cursor.fetchall()        
    except mysql.connector.Error as err:
        print(str(err))
    else:
        cursor.close()
        cnx.close()

    records = []
    for r in records_raw:
        records.append((r[0],r[1]))
        
    return render_template("word_search_c.html", **{"records":records,"records_n":len(records),"word":search_term[2:].decode("hex"),"cluster":search_term[0]})

app.run(host='localhost', port=8080)

 * Running on http://localhost:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [23/May/2018 14:18:06] "GET /cluster_home HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2018 14:18:33] "GET /cluster_home HTTP/1.1" 200 -


In [ ]:
#Could do:

#Home page:
#Make charts on home clickable through to word search

#Live tables:
#Make pie chart click through to cluster (very difficult)

#Need to make a business events page?